In [1]:
!wget -O fma_small.zip https://os.unil.cloud.switch.ch/fma/fma_small.zip
!wget https://os.unil.cloud.switch.ch/fma/fma_metadata.zip

--2023-04-28 18:01:17--  https://os.unil.cloud.switch.ch/fma/fma_small.zip
Resolving os.unil.cloud.switch.ch (os.unil.cloud.switch.ch)... 86.119.28.16, 2001:620:5ca1:201::214
Connecting to os.unil.cloud.switch.ch (os.unil.cloud.switch.ch)|86.119.28.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 7679594875 (7.2G) [application/zip]
Saving to: ‘fma_small.zip’

fma_small.zip       100%[===================>]   7.15G  15.4MB/s    in 8m 3s   

2023-04-28 18:09:21 (15.2 MB/s) - ‘fma_small.zip’ saved [7679594875/7679594875]

--2023-04-28 18:09:21--  https://os.unil.cloud.switch.ch/fma/fma_metadata.zip
Resolving os.unil.cloud.switch.ch (os.unil.cloud.switch.ch)... 86.119.28.16, 2001:620:5ca1:201::214
Connecting to os.unil.cloud.switch.ch (os.unil.cloud.switch.ch)|86.119.28.16|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 358412441 (342M) [application/zip]
Saving to: ‘fma_metadata.zip’

fma_metadata.zip    100%[===================>] 341.81

In [2]:
import zipfile
from tqdm import tqdm

with zipfile.ZipFile('fma_small.zip', 'r') as zip_ref:
    # Get the total number of files to be extracted
    total_files = len(zip_ref.infolist())
    with tqdm(total=total_files, desc='Extracting files') as pbar:
        for file in zip_ref.infolist():
            zip_ref.extract(file)
            pbar.update()

with zipfile.ZipFile('fma_metadata.zip', 'r') as zip_ref:
    # Get the total number of files to be extracted
    total_files = len(zip_ref.infolist())
    with tqdm(total=total_files, desc='Extracting files') as pbar:
        for file in zip_ref.infolist():
            zip_ref.extract(file)
            pbar.update()

Extracting files: 100%|██████████| 12/12 [01:01<00:00,  5.15s/it]


In [4]:
import os
import shutil
import pandas as pd
from tqdm import tqdm

# Load metadata
tracks = pd.read_csv('fma_metadata/tracks.csv', index_col=0, header=[0, 1])

# Filter top genres
genres = ['Hip-Hop', 'Rock', 'Pop', 'Folk', 'Experimental', 'Electronic', 'Instrumental', 'International']
tracks_top_genres = tracks[tracks[('track', 'genre_top')].isin(genres)]

# Create directories for each genre
os.makedirs('fma_filtered', exist_ok=True)
for genre in genres:
    os.makedirs(f'fma_filtered/{genre}', exist_ok=True)

# Number of songs per genre to copy
x = 10

# Counter for songs copied per genre
counter = {genre: 0 for genre in genres}

# Source directory
src_dir = 'fma_small'

# Iterate over subdirectories in fma_small
for subdir in os.listdir(src_dir):
    # Check if it's a directory
    if os.path.isdir(os.path.join(src_dir, subdir)):
        # Iterate over songs in each subdirectory
        for song in os.listdir(os.path.join(src_dir, subdir)):
            # Check if song exists in metadata
            song_id = int(song.split('.')[0])
            if song_id in tracks_top_genres.index:
                # Get the genre of the song
                genre = tracks_top_genres.loc[song_id][('track', 'genre_top')]
                # Check if we've already copied enough songs of this genre
                if counter[genre] < x:
                    # Copy the song to the corresponding genre folder
                    shutil.copy(os.path.join(src_dir, subdir, song), f'fma_filtered/{genre}/{str(counter[genre]).zfill(4)}_{genre}.mp3')
                    # Update the counter
                    counter[genre] += 1

# Print the counter to check how many songs were copied for each genre
print(counter)

{'Hip-Hop': 10, 'Rock': 10, 'Pop': 10, 'Folk': 10, 'Experimental': 10, 'Electronic': 10, 'Instrumental': 10, 'International': 10}


In [9]:
import os
import shutil
import pandas as pd
import numpy as np
import librosa
import librosa.display
import matplotlib.pyplot as plt
from tqdm import tqdm
from scipy.ndimage import zoom

# Create directories for spectrograms
os.makedirs('fma_spectrograms', exist_ok=True)
for genre in genres:
    os.makedirs(f'fma_spectrograms/{genre}', exist_ok=True)

# Convert audio files to spectrograms and save them in the 'fma_spectrograms' directory
for genre in genres:
    for song in os.listdir(f'fma_filtered/{genre}'):
        # Load the audio file
        y, sr = librosa.load(f'fma_filtered/{genre}/{song}')
        
        # Compute the spectrogram
        D = librosa.amplitude_to_db(np.abs(librosa.stft(y)), ref=np.max)

        # Resize the spectrogram
        resize_shape = (224, 224)
        D_resized = zoom(D, (resize_shape[0]/D.shape[0], resize_shape[1]/D.shape[1]))

        # Plot and save the spectrogram
        plt.figure()
        plt.imshow(D_resized, aspect='auto', cmap='viridis')

        # Remove axes
        plt.axis('off')

        plt.savefig(f'fma_spectrograms/{genre}/{song.split(".")[0]}.png', bbox_inches='tight', pad_inches = 0)
        plt.close()